<a href="https://colab.research.google.com/github/eINDUSTRIFY-github/category_match/blob/main/Categorization_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code reads from two files: the first one is external supplier's categories, the second one is eIndustrify's most updated version of categories. It then compares the similarity of categories and match them, if the similarity score is below 0.4: meaning those are not great match, then the program will try to locate all sub/subsub categories within eindustrify's main category and match from there, despite low similarity score. This will make sure at least main category is correct. First cell is the program, second cell is translated PHP.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Specify the path for the input files
eindustrify_path = 'Newly Formed Categories V4.xlsx'
turtle_products_path = 'Turtle products details.xlsx'

def generate_list_from_excel(file_path):
    # Load the Excel file
    excel_file = pd.ExcelFile(file_path)
    data = []

    # Iterate over all sheets in the Excel file
    for sheet_name in excel_file.sheet_names:
        sheet_df = excel_file.parse(sheet_name)

        # Iterate over rows and columns to construct the data list
        for row_index, row in sheet_df.iterrows():
            for col_index, value in row.items():
                data.append({'Sheet Name': sheet_name, 'Column Name': col_index, 'Value': value})
    # Create a DataFrame from the data list
    df = pd.DataFrame(data)
    # Drop rows with NaN values in the 'Value' column
    df.dropna(subset=['Value'], inplace=True)
    # Create a new column 'Combined' by concatenating 'Sheet Name', 'Column Name', and 'Value'
    df['Combined'] = df.apply(lambda x: f"{x['Sheet Name']}/ {x['Column Name']}/ {x['Value']}", axis=1)
    # Keep only the 'Combined' column
    df = df[['Combined']]
    return df

# Read the Excel files
extracted_categories_df = generate_list_from_excel(eindustrify_path)
turtle_products_df = pd.read_excel(turtle_products_path)

# Extract the category lists from the DataFrames
extracted_categories = extracted_categories_df.iloc[:, 0].tolist()
turtle_products = turtle_products_df.iloc[:, 0].tolist()

# Function to compute weighted cosine similarity between two lists of text
def compute_weighted_cosine_similarity(vectorizer, extracted_categories, turtle_products):
    # Transform the categories and products into TF-IDF vectors
    extracted_vectors = vectorizer.transform(extracted_categories)
    turtle_vectors = vectorizer.transform(turtle_products)

    # Compute the cosine similarity matrix
    similarity_matrix = cosine_similarity(turtle_vectors, extracted_vectors)

    return similarity_matrix

# Initialize the vectorizer and compute the similarity matrix
vectorizer = TfidfVectorizer().fit(extracted_categories + turtle_products)
similarity_matrix = compute_weighted_cosine_similarity(vectorizer, extracted_categories, turtle_products)

# Find the best matches for each Turtle product based on cosine similarity
matches = []
for i, turtle_product in enumerate(turtle_products):
    best_match_index = similarity_matrix[i].argmax()  # Find index of the highest similarity score
    best_match_score = similarity_matrix[i, best_match_index]  # Get the highest similarity score
    matches.append((turtle_product, extracted_categories[best_match_index], best_match_score))

# Adjust matches with similarity score less than 0.4 by considering the main category
adjusted_matches = []
for i, (turtle_product, matched_category, similarity_score) in enumerate(matches):
    if similarity_score < 0.4:  # Threshold for adjusting matches
        main_category = turtle_product.split()[0].lower()  # Extract the main category from the product name
        # Filter extracted categories to only those matching the main category
        filtered_extracted_categories = [cat for cat in extracted_categories if cat.split('/')[0].strip().lower() == main_category]
        if filtered_extracted_categories:
            filtered_vectors = vectorizer.transform(filtered_extracted_categories)  # Transform filtered categories into vectors
            # Compute new similarity matrix for filtered categories
            new_similarity_matrix = cosine_similarity(vectorizer.transform([turtle_product]), filtered_vectors)
            best_filtered_match_index = new_similarity_matrix[0].argmax()  # Find index of the highest similarity score in the new matrix
            matched_category = filtered_extracted_categories[best_filtered_match_index]  # Update matched category
            similarity_score = new_similarity_matrix[0, best_filtered_match_index]  # Update similarity score
    adjusted_matches.append((turtle_product, matched_category, similarity_score))

# Convert adjusted matches to DataFrame for saving to Excel
adjusted_matches_df = pd.DataFrame(adjusted_matches, columns=['Turtle Product', 'Matched Category', 'Similarity Score'])

# Save the DataFrame to an Excel sheet
output_path = '/content/Adjusted_Matches.xlsx'
adjusted_matches_df.to_excel(output_path, index=False)

# Download the file automatically
from google.colab import files
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Below is the PHP code